In [ ]:
from sentinelhub import SHConfig
import h5py
import numpy as np
import torch
import datetime
import os
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np


from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    ResamplingType
)

In [ ]:
def create_config(instance_id='8b5b5797-c269-42a4-b75a-dc1ed7e469a8',
                  sh_client_id='f00771eb-624d-4106-973a-98963b395356',
                  sh_client_secret='vnUy3ozu_s[z}W1O/>l6%U6GsfLfs]tLeS@jBs%!',
                  sh_base_url='https://services.sentinel-hub.com',
                  ):
    config = SHConfig(
        instance_id=instance_id,
        sh_client_id=sh_client_id,
        sh_client_secret=sh_client_secret,
        sh_base_url=sh_base_url,
    )
    if not config.sh_client_id or not config.sh_client_secret:
        print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")
    return config
def create_bbox(coordinates:tuple, resolution:int=10, crs:int=32632):
    resolution = resolution
    bbox = BBox(bbox=coordinates, crs=crs)
    bbox_size = bbox_to_dimensions(bbox, resolution=resolution)
    print(f"Image shape at {resolution} m resolution: {bbox_size} pixels")
    return bbox, bbox_size

def create_time_slots(n_chunks :int = 365,
                      start_date :datetime.datetime = datetime.datetime(2022, 1, 1),
                      end_date :datetime.datetime = datetime.datetime(2022, 12, 31)):
    tdelta = (end_date - start_date) / n_chunks
    edges = [(start_date + i * tdelta).date().isoformat() for i in range(n_chunks)]
    slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

    print("time windows:\n")
    for slot in slots:
        print(slot)
    return slots

def get_rgbnir_color_request(time_interval, coordinates:tuple):

    evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "B08"]
            }],
            output: {
                bands: 4
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02 , sample.B08];
    }
"""
    bbox, bbox_size = create_bbox(coordinates)
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=bbox_size,
        config=create_config(),
    )


def get_coords():
    coords = pd.read_csv(r'bboxes_sentinel.csv',index_col=None)
    coords = coords[coords['area'] >0].sort_values(by='area',ascending=False)
    coords= coords.iloc[:,1:5]
    return coords

In [ ]:
config=create_config()

In [ ]:
coords = get_coords().iloc[0:2]
coords

In [ ]:
%%capture download_log --no-stderr
print(f"Start Download at: {datetime.datetime.today()}")

coords = [((361630.678100406, 5140066.039024595, 360630.678100406, 5140166.039024595)),((361630.678100406, 5140066.039024595, 360630.678100406, 5140166.039024595))]
temp_results_tensor = []
# for i,coordinates in coords.iterrows():
for i,coordinates in enumerate(coords):
    print(f'started downloading bounding_box: {coordinates} with index:{i}')
    list_of_requests = [get_rgbnir_color_request(time_interval, tuple(coordinates)) for time_interval in create_time_slots()]
    list_of_requests = [request.download_list[0] for request in list_of_requests]
    # download data with multiple threads
    download_data_timesteps = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)
    non_zero_downloads = torch.tensor([v for i,v in enumerate(download_data_timesteps) if download_data_timesteps[i].sum() > 0]).unsqueeze(0)
    temp_results_tensor.append(non_zero_downloads)

result_tensor = torch.cat(temp_results_tensor, dim=0)
print(f"Stopped Download at: {datetime.datetime.today()}")

print(f"Start Saving Data at: {datetime.datetime.today()}")
file_name = "data.hd5"
with h5py.File(file_name, 'w') as hf:
    dataset_name = 'data'
    hf.create_dataset(dataset_name, data=result_tensor)

print(f"Stopped Saving Data at: {datetime.datetime.today()}")

In [ ]:
with open(r'../_log/download_log.log', 'w') as f:
    f.write(str(download_log))

In [ ]:
file_name = "bern_bbox_2.hd5"  # Replace with the name of your HDF5 file
# Open the HDF5 file in read mode
with h5py.File(file_name, "r") as file:
    # Check if the "data" dataset exists in the file
    if "data" in file:
        # Access the dataset and read its contents into a NumPy array
        dataset = file["data"][:]
    else:
        print("Dataset 'data' not found in the HDF5 file.")


In [ ]:
dataset.shape

In [ ]:
for i in range(dataset.shape[0]):
    for j in range(dataset.shape[1]):
        image = torch.tensor(dataset[i, j,:, :, 0:3])
        
        plt.imshow(image.permute(0, 1, 2).numpy())
        plt.show()

In [ ]:
image.shape

In [ ]:
# Select the image at index 0
image = torch.tensor(dataset[0][ 1, :, :, 0])
image = image.view(240, 240)
# Plot the image
plt.imshow(image.numpy(), cmap='gray')
plt.show()


In [ ]:
# Select the image at index 0
image = torch.tensor(dataset['data'][0][ 0, :, :, 0])
image = image.view(10, 100)
# Plot the image
plt.imshow(image.numpy(), cmap='gray')
plt.show()
